# Pruebas para verificar el cumplimiento del objetivo B

## Configuración Inicial y Autenticación

In [1]:
from utils import configure_headers


# configure the headers
configure_headers()

ENDPOINTS_TO_TEST = [
    # search
    "https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album",
    # get album
    "https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy",
    # get artist
    "https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg",
    # get playlist
    "https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n",
    # get track
    "https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl",
    # TODO Puedes añadir más endpoints aquí
]

✅ Token loaded.


# Objetivo B - Prueba de latencia con VPN

Hacemos las pruebas base para cumplir con el objetivo B, usando la VPN y conexión a internet por fibra. Vamos a probar todos los endpoints que declaramos que son los servicios a estudiar, y vamos a tomar todas las mediciones para luego poder responder a las metricas establecidas.

In [2]:
from utils import simulate_concurrent_users


simulate_concurrent_users(
    num_users=30,
    num_requests_per_user=30,
    endpoints_to_test=ENDPOINTS_TO_TEST
)


--- Simulation started with 30 concurrent users ---
Each user will do 30 requests.

✴️ Usuario 1 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1748095343.3361819
✴️ Usuario 2 - Starting requests to https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy. - Test started at 1748095343.386584
✴️ Usuario 3 - Starting requests to https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg. - Test started at 1748095343.4664094
✴️ Usuario 4 - Starting requests to https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n. - Test started at 1748095343.5485604
✴️ Usuario 5 - Starting requests to https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl. - Test started at 1748095343.6403112
✴️ Usuario 6 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1748095343.6751604
✴️ Usuario 7 - Starting requests to 

## Procesa los resultados

In [3]:
from utils import process_concurrent_results

# process the results
df_global_results, df_individual_results, df_by_endpoint = process_concurrent_results()


--- Resumen General de la Simulación Concurrente ---
Total de usuarios simulados: 30
Peticiones totales realizadas: 900
Peticiones exitosas totales: 480
Errores 4xx totales: 420
Errores 5xx totales: 0
Tasa de éxito global: 53.33%
Tasa de error 4xx global: 46.67%
Tasa de error 5xx global: 0.00%
Latencia promedio: 0.396


In [4]:
print('\nGlobal results\n')
df_global_results


Global results



,user_id,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,total_response_time,latencies,errors_occurred,test_duration
0,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,18,12,0,7.523928,"[0.23353791236877441, 0.2243502140045166, 0.27...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",37.544096
1,8,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,16,14,0,7.476108,"[0.2490711212158203, 0.28029704093933105, 0.23...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",37.497798
2,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,18,12,0,8.460753,"[0.23289775848388672, 0.24607515335083008, 0.2...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.482024
3,15,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,16,14,0,8.130133,"[0.23290324211120605, 0.28166866302490234, 0.3...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.152244
4,20,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,15,15,0,8.382836,"[0.21950030326843262, 0.7343673706054688, 0.27...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.404712
5,19,https://api.spotify.com/v1/playlists/3cEYpjA9o...,30,18,12,0,8.490499,"[0.29065608978271484, 0.2747001647949219, 0.59...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.512676
6,13,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,17,13,0,9.701551,"[0.2911052703857422, 0.24904704093933105, 0.69...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",39.722139
7,23,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,17,13,0,9.187716,"[0.23827099800109863, 0.3010272979736328, 0.24...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",39.208093
8,21,https://api.spotify.com/v1/search?q=remaster%2...,30,15,15,0,9.796643,"[0.4180936813354492, 0.39406633377075195, 0.37...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",39.821673
9,17,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,15,15,0,10.107500,"[0.28342461585998535, 0.25426745414733887, 0.2...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",40.128123


In [5]:
print('\nIndividual results\n')
df_individual_results


Individual results



,user_ids,endpoints,status_codes,latencies,times,error_intervals
0,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.233538,1.748095e+09,NaN
1,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.224350,1.748095e+09,NaN
2,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.279008,1.748095e+09,NaN
3,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.228345,1.748095e+09,NaN
4,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.220560,1.748095e+09,NaN
...,...,...,...,...,...,...
895,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,200,1.421414,1.748095e+09,NaN
896,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,200,0.257079,1.748095e+09,NaN
897,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,200,0.262588,1.748095e+09,NaN
898,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,200,0.348448,1.748095e+09,NaN


In [6]:
print('\nResults by endpoint\n')
df_by_endpoint


Results by endpoint



,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,avg_latency,num_users,success_rate,error_4xx_rate,error_5xx_rate
0,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,180,95,85,0,0.482927,6,52.777778,47.222222,0.0
1,https://api.spotify.com/v1/artists/0TnOYISbd1X...,180,98,82,0,0.326937,6,54.444444,45.555556,0.0
2,https://api.spotify.com/v1/playlists/3cEYpjA9o...,180,97,83,0,0.400510,6,53.888889,46.111111,0.0
3,https://api.spotify.com/v1/search?q=remaster%2...,180,93,87,0,0.454234,6,51.666667,48.333333,0.0
4,https://api.spotify.com/v1/tracks/11dFghVXANMl...,180,97,83,0,0.316912,6,53.888889,46.111111,0.0


## Guardamos los resultados base

In [7]:
df_global_results.to_csv('./files/obj-B/global_results.csv', index=False)
df_individual_results.to_csv('./files/obj-B/individual_results.csv', index=False)
df_by_endpoint.to_csv('./files/obj-B/endpoints_results.csv', index=False)